## Kafka Ingestion

### Required Imports + Subscribing to topic

In [1]:
from confluent_kafka import Consumer
import uuid
import os
from dotenv import load_dotenv
import snowflake.connector
import os
load_dotenv()

KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_USERNAME=os.getenv('KAFKA_USERNAME')
KAFKA_PASSWORD=os.getenv('KAFKA_PASSWORD')
KAFKA_TOPIC_NAME = os.getenv('KAFKA_TOPIC_NAME')

values = []

# Consumer with `confluent_kafka`
c = Consumer({
    'bootstrap.servers': KAFKA_SERVER,
    'group.id': f'deleton' +str(uuid.uuid1()),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_USERNAME,
    'sasl.password': KAFKA_PASSWORD,
    'session.timeout.ms': 6000,
    'heartbeat.interval.ms': 1000,
    'fetch.wait.max.ms': 6000,
    'auto.offset.reset': 'latest',
    'enable.auto.commit': 'false',
    'max.poll.interval.ms': '86400000',
    'topic.metadata.refresh.interval.ms': "-1",
    "client.id": 'id-002-005',
})

c.subscribe([KAFKA_TOPIC_NAME])
"""Kafka consumer ready and consumes messages"""

/opt/homebrew/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


'Kafka consumer ready and consumes messages'

In [83]:
def extract_values_from_log(string):
    regex=r'\d+.?\d+|\d'
    log_values = re.findall(regex,string)
    return log_values

### Testing Kafka Messages

In [120]:
import json
import re
i = 0
no_system_log = True
while no_system_log:
    kafka_message = c.poll(0.5) #poll all messages

    if kafka_message is not None: #exclude none values
        log = kafka_message.value().decode('utf-8')

        if 'INFO' in log: #only check for strings with INFO
            values = json.loads(log)
            new_log = values.get('log')
         

            if 'Ride' in new_log: #process strings with Ride info
                new_log = new_log.split(' mendoza v9: [INFO]: Ride - ')
            
                timestamp = new_log[0]
               
                log_values = extract_values_from_log(new_log[1])
                duration=int(float(log_values[0]))
                resistance=log_values[1]
         

           
            if 'Telemetry' in new_log:
                new_log = new_log.split(' mendoza v9: [INFO]: Telemetry - ')
               
                timestamp = new_log[0]
                log_values = extract_values_from_log(new_log[1])
                heart_rate = log_values[0]
                rpm = log_values[1]
                power= round(float(log_values[2]),3)

        
        if 'SYSTEM' in log:
            system_log = json.loads(log).get('log')
            split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
            timestamp = split_log[0]
            dictionary_string = split_log[1][:-1]
            user_dictionary = json.loads(dictionary_string)
            print(user_dictionary)


    i += 1

{'user_id': 4563, 'name': 'Ms Jodie Dale', 'gender': 'female', 'address': '74 Robert wall,Port James,B9S 4DS', 'date_of_birth': 329184000000, 'email_address': 'ms.jodie28@yahoo.com', 'height_cm': 163, 'weight_kg': 55, 'account_create_date': 1643932800000, 'bike_serial': 'SN0000', 'original_source': 'offline'}


KeyboardInterrupt: 

In [157]:
# system_log = json.loads(system_log).get('log')
split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
timestamp = split_log[0]
dictionary_string = split_log[1][:-1]
user_dictionary = json.loads(dictionary_string)
user_dictionary

{'user_id': 4563,
 'name': 'Ms Jodie Dale',
 'gender': 'female',
 'address': '74 Robert wall,Port James,B9S 4DS',
 'date_of_birth': 329184000000,
 'email_address': 'ms.jodie28@yahoo.com',
 'height_cm': 163,
 'weight_kg': 55,
 'account_create_date': 1643932800000,
 'bike_serial': 'SN0000',
 'original_source': 'offline'}

In [158]:
from datetime import datetime,date

def convert_unix_to_date(unix_timestamp):
    unix_timestamp /= 1000  #convert to seconds as unix timestamp is in milliseconds
    converted_to_date = datetime.utcfromtimestamp(unix_timestamp) 
  
    current_time = datetime.now()
    if converted_to_date > current_time:
        return 'N/A'

 
    return converted_to_date.strftime('%Y-%m-%d')

user_dictionary['account_create_date'] = convert_unix_to_date(user_dictionary['account_create_date'])
user_dictionary['date_of_birth'] = convert_unix_to_date(user_dictionary['date_of_birth'])

user_dictionary

{'user_id': 4563,
 'name': 'Ms Jodie Dale',
 'gender': 'female',
 'address': '74 Robert wall,Port James,B9S 4DS',
 'date_of_birth': '1980-06-07',
 'email_address': 'ms.jodie28@yahoo.com',
 'height_cm': 163,
 'weight_kg': 55,
 'account_create_date': '2022-02-04',
 'bike_serial': 'SN0000',
 'original_source': 'offline'}

### TODO: 

* Read kafka data into a dataframe (Pandas or PySpark?)
* Write data into Snowflake

In [ ]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
SCHEMA= os.environ.get('SCHEMA')

In [ ]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema='ZOOKEEPERS_BATCH_PRODUCTION'
)
cs = conn.cursor()
"""Already created schema to use"""

In [ ]:
query = "SHOW TABLES"
result = cs.execute(query)
result.fetchmany(100)

[]